## Pima Indians Diabetes

Our objective is to predict whether a patient is suffering or not form diabetes. 

Our input values include "number of pregnancies the patient has had, their BMI, insulin level, age" and others.
Our ouput value will be a boolean, i.e 1 if he has diabetes and 0 if not.

### Loading of packages and dataset path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data Pre-Processing

In [ ]:
#we load the dataset
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.shape

We have 768 rows in our dataset with 9 columns (8 predictor variables and 1 outcome variable)

In [ ]:
df.columns

In [ ]:
df.describe()

### N/A Values

In [ ]:
df.isna().sum()

We don't have any N/A value

## Machine Learning

### X/Y definition

In [ ]:
X = df.drop(['Outcome'], axis = 1)
y = df.Outcome

### Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Testing different models

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

classifiers = [
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    LogisticRegression()
    ]


for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
print("="*30)

### Baselines

To conclude on the accuracy of our models, we need to compare them with different baselines.

These baselines will be given by the Dummy Classifier.

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dc_strat = DummyClassifier(strategy = "stratified")
dc_strat.fit(X_train, y_train)

dc_freq = DummyClassifier(strategy = "most_frequent")
dc_freq.fit(X_train, y_train)

print("DC Stratified accuracy : ", dc_strat.score(X_test, y_test))
print("DC Most Frequent accuracy : ", dc_freq.score(X_test, y_test))

### Parameters tuning : SVC

We want to improve the accuracy we have with our classifiers. To do so, we can tune their parameters.

Let's pick our SVC model. We will tune its paramters thanks to the Cross-Validation tool of Grid Search.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Set the parameters by cross-validation
parameters = {'kernel': ['rbf'],  'C': [1, 10, 100, 1000]}


clf = GridSearchCV(SVC(), parameters)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Best score:")
print()
print(clf.best_score_)

## Problème score GridSearchCV

<span style="color:red"> La précision du modèle avec hyperparamètres optimisés par GridSearch CV est moins bonne que la précision du modèle à vide... cf question posée en cours, je ne vois pas d'où peut venir le problème. Dans la documentation du SVC, la valeur par défaut de C est 1, et la valeur de kernel est 'rbf', que j'ai bien inclus dans la liste des hyperparamètres à tester.